In [1]:
import os
SPARK_HOME = '/Users/isang-geon/spark'
# exec(open(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py')).read())
exec(open(os.path.join(SPARK_HOME, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.0.1
      /_/

Using Python version 3.8.5 (default, Sep  4 2020 02:22:02)
SparkSession available as 'spark'.


In [3]:
from pyspark.ml.linalg import DenseVector
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql import Row

# 레이블이 지정된 관찰의 데이터프레임 준비
outlook = {"sunny": 0.0, "overcast": 1.0, "rainy": 2.0}
observations = [
    Row(label=0, features=DenseVector([outlook["sunny"], 85, 85, False])),
    Row(label=0, features=DenseVector([outlook["sunny"], 80, 90, True])),
    Row(label=1, features=DenseVector([outlook["overcast"], 83, 96, False])),
    Row(label=1, features=DenseVector([outlook["rainy"], 70, 96, False])),
    Row(label=1, features=DenseVector([outlook["rainy"], 68, 80, False])),
    Row(label=0, features=DenseVector([outlook["rainy"], 65, 70, True])),
    Row(label=1, features=DenseVector([outlook["overcast"], 64, 65, True])),
    Row(label=0, features=DenseVector([outlook["sunny"], 72, 95, False])),
    Row(label=1, features=DenseVector([outlook["sunny"], 69, 70, False])),
    Row(label=1, features=DenseVector([outlook["sunny"], 75, 80, False])),
    Row(label=1, features=DenseVector([outlook["sunny"], 75, 70, True])),
    Row(label=1, features=DenseVector([outlook["overcast"], 72, 90, True])),
    Row(label=1, features=DenseVector([outlook["overcast"], 81, 75, False])),
    Row(label=0, features=DenseVector([outlook["rainy"], 71, 91, True]))
]
rdd = sc.parallelize(observations)
data = spark.createDataFrame(rdd)

In [4]:
# 데이터를 교육 및 테스트 세트로 분할
(trainingData, testData) = data.randomSplit([0.7, 0.3])

In [6]:
# 의사 결정 트리 모델 교육
dt = DecisionTreeClassifier()
model = dt.fit(trainingData)

In [7]:
# 테스트 데이터 세트를 사용해 예측하기
predictions = model.transform(testData)
predictions.show()

+-----+-------------------+-------------+-----------+----------+
|label|           features|rawPrediction|probability|prediction|
+-----+-------------------+-------------+-----------+----------+
|    0|[0.0,85.0,85.0,0.0]|    [2.0,0.0]|  [1.0,0.0]|       0.0|
|    1|[2.0,68.0,80.0,0.0]|    [1.0,0.0]|  [1.0,0.0]|       0.0|
|    1|[2.0,70.0,96.0,0.0]|    [1.0,0.0]|  [1.0,0.0]|       0.0|
|    1|[0.0,69.0,70.0,0.0]|    [0.0,1.0]|  [0.0,1.0]|       1.0|
|    1|[0.0,75.0,80.0,0.0]|    [2.0,0.0]|  [1.0,0.0]|       0.0|
|    0|[2.0,71.0,91.0,1.0]|    [1.0,0.0]|  [1.0,0.0]|       0.0|
+-----+-------------------+-------------+-----------+----------+



In [8]:
# 모델 정확성 평가
evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g " %(1.0 - accuracy))

Test Error = 0.5 
